In [3]:
import pathlib as Path 
print(Path.Path.cwd())

/home/manimala/Documents/satyakama/paper-farmer-chatbot


In [4]:
import polars as pl

# Test if we can create a simple DataFrame
try:
    print("Testing Polars version:", pl.__version__)
    test_df = pl.DataFrame({"a": [1, 2, 3]})
    print("Basic DataFrame creation works:", test_df)
except Exception as e:
    print("Error:", e)

Testing Polars version: 1.21.0
Basic DataFrame creation works: shape: (3, 1)
┌─────┐
│ a   │
│ --- │
│ i64 │
╞═════╡
│ 1   │
│ 2   │
│ 3   │
└─────┘


In [10]:
import polars as pl

try:
    df = pl.read_csv('/home/manimala/Documents/satyakama/paper-farmer-chatbot/dataset/processed_dataset/India_level_data_filtered.csv')
    print("Basic read_csv works")
except Exception as e:
    print("Error:", e)

Basic read_csv works


In [11]:
import polars as pl
import psutil
import gc
import os
from typing import Optional, Union, Dict, List

def clear_memory(df_to_remove: Optional[Union[pl.DataFrame, List[pl.DataFrame]]] = None) -> None:
    """
    Clear memory and print memory usage statistics
    """
    try:
        # Print initial state
        print("\nInitial memory state:")
        process = psutil.Process(os.getpid())
        initial_memory_mb = process.memory_info().rss / 1024 / 1024
        print(f"Current Memory Usage: {initial_memory_mb:.2f} MB ({initial_memory_mb/1024:.2f} GB)")
        
        # Remove specific DataFrame if provided
        if df_to_remove is not None:
            if isinstance(df_to_remove, list):
                for df in df_to_remove:
                    # Get the variable name
                    name = [var_name for var_name, var_val in globals().items() if var_val is df]
                    if name:
                        globals().pop(name[0], None)
                    del df
            else:
                name = [var_name for var_name, var_val in globals().items() if var_val is df_to_remove]
                if name:
                    globals().pop(name[0], None)
                del df_to_remove
                
        # Force garbage collection
        gc.collect()
        
        # Get new memory info
        new_memory_mb = process.memory_info().rss / 1024 / 1024
        memory_freed = initial_memory_mb - new_memory_mb
        print(f"\nCurrent Memory Usage: {new_memory_mb:.2f} MB ({new_memory_mb/1024:.2f} GB)")
        print(f"Available System Memory: {psutil.virtual_memory().available / 1024 / 1024 / 1024:.2f} GB")
        print(f"Memory Utilization: {psutil.virtual_memory().percent}%")
        if memory_freed > 0:
            print(f"Memory freed: {memory_freed:.2f} MB")
    except Exception as e:
        print(f"Memory cleanup failed: {e}")

def load_data(
    file_path: str,
    schema_overrides: Optional[Dict] = None,
    columns_to_drop: Optional[List[str]] = None,
    low_memory: bool = True
) -> Optional[pl.DataFrame]:
    """
    Load and process a CSV file using Polars
    """
    try:
        # Ensure file exists
        if not Path(file_path).exists():
            raise FileNotFoundError(f"File not found: {file_path}")
            
        print("Initial memory state:")
        clear_memory()
        
        print("\nLoading CSV file...")
        # Read CSV with provided schema overrides
        df = pl.read_csv(
            file_path,
            schema_overrides=schema_overrides or {},
            low_memory=low_memory
        )
        
        # Drop specified columns if any
        if columns_to_drop:
            df = df.drop(columns_to_drop)
            
        print("\nAfter loading CSV:")
        clear_memory()
        return df
        
    except Exception as e:
        print(f"Error in data processing: {e}")
        return None

# Your schema and column definitions remain the same
schema = {
    'Year': pl.Int32,
    'Month': pl.Int32,
    'Day': pl.Int32,
    'Crop': pl.Utf8,
    'DistrictName': pl.Utf8,
    'QueryType': pl.Utf8,
    'Season': pl.Utf8,
    'Sector': pl.Utf8,
    'StateName': pl.Utf8,
    'QueryText': pl.Utf8,
    'KccAns': pl.Utf8,
    'Category': pl.Utf8,
    'BlockName': pl.Utf8,
    
}

# Specify columns to drop
columns_to_drop = ['BlockName', 'Category']

# Load your data
master_df = load_data(
    file_path='home/manimala/Documents/satyakama/paper-farmer-chatbot/dataset/processed_dataset/India_level_data_filtered.csv',
    schema_overrides=schema,
    columns_to_drop=columns_to_drop,
    low_memory=True
)

Error in data processing: 'module' object is not callable
